# 1. rubot Mecanum model description

Diferent models could be obtained:
- Commercial robot nexus robot
- Costomized nexus robot
- Custom designed model 

For a custom model created in SolidWorks, there is an add-in that allows the conversion of SW Parts and Assemblies into a URDF file. The exporter will create a ROS-like package that contains a directory for meshes, textures and robots (urdf files).: http://wiki.ros.org/sw_urdf_exporter




## 1.1. Commercial nexus robot

This commercial robot can be found in: https://www.nexusrobot.com/product/4wd-mecanum-wheel-mobile-arduino-robotics-car-10011.html

There is already a URDF model extracted from: https://github.com/RBinsonB/nexus_4wd_mecanum_simulator

We have taken the nexus_4wd_mecanum_description package from the RBinsonB repository

Launch the nexus_4wd_mecanum_description.launch file has to include the "joint_state_publisher" node. This correction has to be done:


In [ ]:
<?xml version="1.0"?>
<launch>
	<param name="robot_description" command="$(find xacro)/xacro --inorder $(find nexus_4wd_mecanum_description)/urdf/nexus_4wd_mecanum.xacro" />
    <node name="joint_state_publisher" pkg="joint_state_publisher" type="joint_state_publisher" />
    <node name="robot_state_publisher" pkg="robot_state_publisher" type="robot_state_publisher" />
</launch>


Run the simulation for one nexus robot by using the following command line:

In [ ]:
roslaunch nexus_4wd_mecanum_description nexus_4wd_mecanum_rviz.launch 

![Getting Started](./Images/1_nexus_4wd.png)

## 1.2. Costomized nexus robot

We have created a new "nexus_mecanum" package in robot description folder.
In the URDF file we have added:
- Camera sensor
- LIDAR sensor
- Kinematic driver for Holonomic movement


### Camera sensor

This sensor is integrated as a link and fixed joint for visual purposes:

In [ ]:
  <!-- 2D Camera as a mesh of actual PiCamera -->
  <link name="camera">
    <visual>
      <origin rpy="0 1.570795 0" xyz="0 0 0"/>
      <geometry>
        <mesh filename="package://nexus_4wd_mecanum_description/meshes/piCamera.stl" scale="0.0025 0.0025 0.0025"/>
      </geometry>
      <material name="yellow"/>
    </visual>
    <collision>
      <origin rpy="0 1.570795 0" xyz="0 0 0"/>
      <geometry>
        <box size="0.075 0.075 0.025"/>
      </geometry>
    </collision>
    <inertial>
      <origin rpy="0 1.570795 0" xyz="0 0 0"/>
      <mass value="1e-3"/>
      <inertia ixx="1e-6" ixy="0" ixz="0" iyy="1e-6" iyz="0" izz="1e-6"/>
    </inertial>
  </link>
  <!-- 2D Camera JOINT base_link -->
  <joint name="joint_camera" type="fixed">
    <axis xyz="0 0 1"/>
    <origin rpy="0 0 0" xyz="0.16 0 0.05"/>
    <parent link="base_link"/>
    <child link="camera"/>
  </joint>

A driver is needed to view the images.

In [ ]:
  <!-- 2D Camera controller -->
  <gazebo reference="camera">
    <sensor name="camera1" type="camera">
      <update_rate>30.0</update_rate>
      <camera name="front">
        <horizontal_fov>1.3962634</horizontal_fov>
        <image>
          <width>800</width>
          <height>800</height>
          <format>R8G8B8</format>
        </image>
        <clip>
          <near>0.02</near>
          <far>300</far>
        </clip>
      </camera>
      <plugin filename="libgazebo_ros_camera.so" name="camera_controller">
        <alwaysOn>true</alwaysOn>
        <updateRate>0.0</updateRate>
        <cameraName>rubot/camera1</cameraName>
        <imageTopicName>image_raw</imageTopicName>
        <cameraInfoTopicName>camera_info</cameraInfoTopicName>
        <frameName>camera</frameName>
        <hackBaseline>0.07</hackBaseline>
        <distortionK1>0.0</distortionK1>
        <distortionK2>0.0</distortionK2>
        <distortionK3>0.0</distortionK3>
        <distortionT1>0.0</distortionT1>
        <distortionT2>0.0</distortionT2>
      </plugin>
    </sensor>
  </gazebo>

### LIDAR sensor

This sensor is integrated as a link and fixed joint for visual purposes:

In [ ]:
  <!-- LIDAR base_scan -->
  <link name="base_scan">
    <visual name="sensor_body">
      <origin rpy="0 0 0" xyz="0 0 0.04"/>
      <geometry>
        <mesh filename="package://nexus_4wd_mecanum_description/meshes/X4.stl" scale="0.0015 0.0015 0.0015"/>
      </geometry>
      <material name="yellow"/>
    </visual>
    <collision>
      <origin rpy="0 0 0" xyz="0 0 0.04"/>
      <geometry>
        <cylinder length="0.01575" radius="0.0275"/>
      </geometry>
    </collision>
    <inertial>
      <origin rpy="0 0 0" xyz="0 0 0.4"/>
      <mass value="0.057"/>
      <inertia ixx="0.001" ixy="0.0" ixz="0.0" iyy="0.001" iyz="0.0" izz="0.001"/>
    </inertial>
  </link>
  <!-- LIDAR base_scan JOINT base_link -->
  <joint name="scan_joint" type="fixed">
    <axis xyz="0 0 1"/>
    <origin rpy="0 0 0" xyz="0 0 0.09"/>
    <parent link="base_link"/>
    <child link="base_scan"/>
  </joint>

A driver is needed to see the 720 laser distance points:

In [ ]:
  <!-- Laser Distance Sensor YDLIDAR X4 controller-->
  <gazebo reference="base_scan">
    <sensor name="lds_lfcd_sensor" type="ray">
      <pose>0 0 0 0 0 0</pose>
      <visualize>false</visualize>
      <update_rate>5</update_rate>
      <ray>
        <scan>
          <horizontal>
            <samples>720</samples>
            <resolution>0.5</resolution>
            <min_angle>0.0</min_angle>
            <max_angle>6.28319</max_angle>
          </horizontal>
        </scan>
        <range>
          <min>0.12</min>
          <max>10</max>
          <resolution>0.015</resolution>
        </range>
        <noise>
          <type>gaussian</type>
          <!-- Noise parameters based on published spec for YDLIDAR X4
              is 1.5% at half range 4m (= 60mm, "+-160mm" accuracy at max. range 8m).
              A mean of 0.0m and stddev of 0.020m will put 99.7% of samples
              within 0.16m of the true reading. -->
          <mean>0.0</mean>
          <stddev>0.02</stddev>
        </noise>
      </ray>
      <plugin filename="libgazebo_ros_laser.so" name="gazebo_ros_lds_lfcd_controller">
        <!-- topicName>/gopigo/scan</topicName -->
        <topicName>scan</topicName>
        <frameName>base_scan</frameName>
      </plugin>
    </sensor>
  </gazebo>

### Mecanum Drive controller
A driver is needed to describe the kinematics.This kinematics is described in the "libgazebo_ros_planar_move.so" file and the URDF model will contain the specific gazebo plugin.

This driver is the "Planar Move Plugin" and is described in Gazebo tutorials: http://gazebosim.org/tutorials?tut=ros_gzplugins#AddingaModelPlugin

In [ ]:
  <!-- Mecanum drive controller -->
  <gazebo>
    <plugin name="Mecanum_controller" filename="libgazebo_ros_planar_move.so">
      <commandTopic>cmd_vel</commandTopic>
      <odometryTopic>odom</odometryTopic>
      <odometryFrame>odom</odometryFrame>
      <odometryRate>50.0</odometryRate>
      <robotBaseFrame>base_footprint</robotBaseFrame>
    </plugin>
  </gazebo>

We can open the new model in rviz and gazebo:

- roslaunch nexus_mecanum display.launch
- roslaunch nexus_mecanum gazebo.launch

![Getting Starter](./Images/1_nexus_mecanum2.png)

To control the POSE with cmd_vel we can install the package: 
- For Holonomic movements: http://wiki.ros.org/teleop_twist_keyboard
- for non-holonomic movements: http://wiki.ros.org/teleop_tools

In [ ]:
sudo apt-get install ros-melodic-teleop-tools
or
sudo apt-get install ros-melodic-teleop-twist-keyboard

To launch the control:

In [ ]:
rosrun key_teleop key_teleop.py /key_vel:=/cmd_vel
or
rosrun teleop_twist_keyboard teleop_twist_keyboard.py

### 1.3. rUBot mecanum model

We can create a new model in 3D using SolidWorks and use the URDF plugin to generate the URDF file model: rubot_mecanum.urdf

This model is located in a new "rubot_mecanum" package

We add the same sensors and plugins.

We can open the new model in rviz and gazebo:

- roslaunch rubot_mecanum display.launch
- roslaunch rubot_mecanum gazebo.launch

![Getting Starter](./Images/1_rubot_mecanum2.png)

# 2. Mecanum robot spawn & control

We have first to design a world where our robot will perform specific navigation tasks

We will use nexus robot model

## 2.1. World definition

Here we have first to design the world, for exemple a maze from where our rUBot mecanum has to navigate autonomously.

There is a very useful and simple tool to design a proper world: "Building editor" in gazebo.

Open gazebo as superuser:

- sudo gazebo

You can build your world using "Building Editor" in Edit menu

![Getting Started](./Images/1_maze1.png)

You can save:

- the generated model in a model folder (without extension)

Close the Builder Editor, modify the model position and add other elements if needed. Save:

- the generated world (with extension .world) in the world folder.

Once you finish is better to close the terminal you have work as superuser

## 2.2. Spawn rubot Mecanum in world
We have created a specific world "square.world" and we can spawn the robot model inside this world.

For that, we have created a "nexus_world.launch" file.

Type to launch the file:

- roslaunch nexus_mecanum nexus_world.launch
- roslaunch nexus_mecanum display.launch

In [ ]:
<?xml version="1.0"?>
<launch>
  <!-- Define the needed parameters -->
  <arg name="world" default="square.world"/> 
  <arg name="model" default="nexus.urdf" />
  <arg name="x_pos" default="0.5"/>
  <arg name="y_pos" default="0.5"/>
  <arg name="z_pos" default="0.0"/>

  <include file="$(find gazebo_ros)/launch/empty_world.launch">
    <arg name="world_name" value="$(find nexus_mecanum)/worlds/$(arg world)" />
  </include>
  <!-- Spawn the robot -->
  <!-- Robot URDF definition -->
    <param name="robot_description" textfile="$(find nexus_mecanum)/urdf/$(arg model)"/>
  <node name="spawn_model" pkg="gazebo_ros" type="spawn_model"
    args="-urdf -model nexus -param robot_description -x $(arg x_pos) -y $(arg y_pos) -z $(arg z_pos)" />
</launch>


In the case of nexus_mecanum robot:

![Getting Starter](./Images/1_nexus_mecanum.png)

To control the robot with the Keyboard you have to install the "teleop-tools" package:

Perhaps is needed to setup your Keys again:

- curl -s https://raw.githubusercontent.com/ros/rosdistro/master/ros.asc | sudo apt-key add -
- sudo apt get update

Then you can control the nexus robot with the following package

In [ ]:
sudo apt-get install ros-melodic-teleop-twist-keyboard

Then you will be able to control the robot with the Keyboard typing:

In [ ]:
rosrun teleop_twist_keyboard teleop_twist_keyboard.py

# 3. rubot mecanum navigation control in the new world environment

Once the world has been generated we will create a ROS Package "rubot_control" to perform the autonomous navigation

In [ ]:
cd ~/rubot_mecanum_ws/src
catkin_create_pkg nexus_control rospy std_msgs sensor_msgs geometry_msgs nav_msgs
cd ..
catkin_make

## 3.1 Kinematics model of mecanum robot

The first concept we are going to go through is kinematic models. So, we know what kinematics are, but, what is a kinematic model?

Wheeled mobile robots may be classified in two major categories, holonomic or (omnidirectional) and nonholonomic. Whether a wheeled mobile robot is omnidirectional or not depends in part on the type of wheels it employs. Nonholonomic mobile robots, such as conventional cars, employ conventional wheels, which prevents cars from moving directly sideways.

We will define the Kinematic model for Holonomic Mecanum wheeled robot

Specifically, you will learn about:

- Kinematic model of a holonomic robot
- Motion with respect to the robot's frame
- Motion with respect to the world's frame

A kinematic model of a mobile robot governs how wheel speeds map to robot velocities. We assume that the robot rolls on hard, flat, horizontal ground without skidding, and it has a single rigid-body chassis (not articulated like a tractor-trailer) with a configuration 𝑇𝑠𝑏∈𝑆𝐸(2) representing a chassis-fixed frame {b} relative to a fixed space frame {s} in the horizontal plane.

Omnidirectional wheeled mobile robots typically employ either omniwheels or mecanum wheels, which are typical wheels augmented with rollers on their outer circumference. These rollers spin freely and they allow sideways sliding while the wheel drives forward or backward without slip in that direction.

Unlike the front wheels in a car, those wheels are not steered, only driven forward or backward.

We represent 𝑇𝑠𝑏 by the three coordinates 𝑞=(𝜙,𝑥,𝑦) , and the velocity of the chassis as the time derivative of the coordinates, 𝑞˙=(𝜙˙,𝑥˙,𝑦˙) . We also define the chassis' planar twist 𝜈𝑏=(𝜔𝑏𝑧,𝑣𝑏𝑥,𝑣𝑏𝑦) expressed in {b}

The kinematic model of the mobile robot with four mecanum wheels is depicted below.

The different movements our car can perform are:

<img src="./Images/1_mecanumDrive.png">

![Getting Started](./Images/1_mecanumDrive1.png)

The kinematics model defining the cartesian velocities in function ot wheel velocities is described:

![Getting Started](./Images/1_mecanumDrive2.png)

where

- Vi: Linear speed of the wheels.
- ωdi: Angular speed of the wheels. (It is equal to Wdi in the calculation of the kinematics)
- Vir: Tangential speed of the rollers.
- ul: Linear velocity of the system on the X axis.
- uf: Linear velocity of the system on the Y axis.
- ω: Speed of rotation of the system on the Z axis.
- a: Distance from the center of the robot to the axis of rotation of the wheel.
- b: Distance from the center of the robot to the center of the width of the wheel.

(see [Lynch & Park, 2017] for a complete derivation of this model).

## 3.2. Mecanum control in a world environment

Diferent navigation programs are created:

- movement control:
    - a python file "rubot_control1.py" to control the movement linear velocity in x-y direction and angular velocity in z directiron and with a maximum displacement of direction
    - a launch file "rubot_control1.launch"
    - The parameters vx,vy,w and d can be configured in the launch file
- Lidar test: to verify the LIDAR readings and angles
- Autonomous navigation: using the LIDAR to avoid obstacles
- follow wall
- go to POSE

### a) Movement control

Control the movement linear velocity in x-y direction and angular velocity in z directiron and with a maximum displacement of direction

In [ ]:
<launch>
    <arg name="world" default="square.world"/> 
    <arg name="model" default="nexus.urdf" />
    <arg name="x_pos" default="0.0"/>
    <arg name="y_pos" default="0.0"/>
    <arg name="z_pos" default="0.0"/>
  
    <include file="$(find gazebo_ros)/launch/empty_world.launch">
      <arg name="world_name" value="$(find nexus_control)/worlds/$(arg world)"/>
    </include>
  
  <!-- Load rviz -->
    <include file="$(find nexus_mecanum)/launch/display.launch" />

  <!-- Spawn the robot -->
    <param name="robot_description" textfile="$(find nexus_mecanum)/urdf/$(arg model)" />
    <node pkg="gazebo_ros" type="spawn_model" name="spawn_model"
      args="-urdf -model rubot -x $(arg x_pos) -y $(arg y_pos) -z $(arg z_pos) -param robot_description" />
  
  <!-- run navigation program   -->
    <arg name="vx" default="0.1"/>
    <arg name="vy" default="0.1"/>
    <arg name="w" default="0.2"/>
    <arg name="d" default="0.5"/>
    <node pkg="nexus_control" type="rubot_control1.py" name="rubot_nav" output="screen" >
      <param name="vx" value="$(arg vx)"/>
      <param name="vy" value="$(arg vy)"/>
      <param name="w" value="$(arg w)"/>
      <param name="d" value="$(arg d)"/>
    </node>
  </launch>

In [ ]:
roslaunch nexus_control rubot_control1.launch

### b) LIDAR test

to verify the LIDAR readings and angles

In [ ]:
#! /usr/bin/env python

import rospy
from sensor_msgs.msg import LaserScan

def callback(msg):
    print ("Number of scan points: "+ str(len(msg.ranges)))
    # values at 0 degrees
    print ("Distance at 0deg: " + str(msg.ranges[0]))
    # values at 90 degrees
    print ("Distance at 90deg: " + str(msg.ranges[180]))
    # values at 180 degrees
    print ("Distance at 180deg: " + str(msg.ranges[360]))
    # values at 270 degrees
    print ("Distance at 270deg: " + str(msg.ranges[540]))
    # values at 360 degrees
    print ("Distance at 360deg: " + str(msg.ranges[719]))

rospy.init_node('scan_values')
sub = rospy.Subscriber('/scan', LaserScan, callback)
rospy.spin()

In [ ]:
roslaunch nexus_control rubot_lidar_test.launch
rosrun teleop_twist_keyboard teleop_twist_keyboard.py

![Getting Started](./Images/1_nexus_lidar_test.png)

### c) Autonomous navigation

Navigate autonomously using the LIDAR to avoid obstacles.

In [ ]:
roslaunch nexus_control rubot_self_nav.launch

![Getting Started](./Images/1_nexus_self.png)

### d) Follow wall

Follow the wall accuratelly to make a map with precision to apply SLAM techniques for navigation purposes 

![Getting Started](./Images/1_nexus_wall_follow.png)

### e) Go to POSE

Define a specific Position and Orientation as a target point to nexus robot